In [96]:
!pip install -q -U zarr
!pip install -q -U fsspec
!pip install -q -U gcsfs
!pip install -q -U xarray
!pip install -q -U dask-ml[complete]  

In [47]:
import xarray as xr
import fsspec

ds = xr.open_zarr(fsspec.get_mapper('gcs://weather-bench/region_zarr/'), consolidated=True)

print("Tamanño en GB:", ds.nbytes / 1e9)

Tamanño en GB: 19.324476644


In [53]:
ds

<xarray.Dataset>
Dimensions:    (lat: 12, level: 13, lon: 14, time: 350640)
Coordinates:
  * lat        (lat) float64 -70.31 -64.69 -59.06 ... -19.69 -14.06 -8.438
  * level      (level) int32 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * lon        (lon) float64 247.5 253.1 258.8 264.4 ... 303.8 309.4 315.0 320.6
  * time       (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
Data variables:
    lat2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lon2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lsm        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    orography  (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    pv         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    q          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    r          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    slt        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    t          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    t2m        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    tcc        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    u10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    v10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    vo         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    z          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-03-08 20:51:24 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

# Linear Regression for $Z500$, $T850$ and $T2M$

In [108]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from score_basic import *

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error # 'neg_root_mean_squared_error'
from dask_ml.wrappers import ParallelPostFit

In [91]:
def load_test_data(var, ds=None, path=None, years=slice('2017', '2018')):
    """
    Load the test dataset. If z return z500, if t return t850.
    Args:
        ds: xarray dataset
        path: Path to nc files
        var: variable. Geopotential = 'z', Temperature = 't'
        years: slice for time window
    Returns:
        dataset: Concatenated dataset for 2017 and 2018
    """
    if (path is None) and (ds is None):
        raise ValueError('Give ds or path')

    if ds is None:
        ds = xr.open_mfdataset(f'{path}/*.nc', combine='by_coords')[var]
        if var in ['z', 't']:
            try:
                ds = ds.sel(level=500 if var == 'z' else 850).drop('level')
            except ValueError:
                ds = ds.drop('level')
    elif path is None:
        ds = ds[var]
        if var in ['z', 't']:
            print('Selecting from ds...')
            try:
                ds = ds.sel(level=500 if var == 'z' else 850).drop('level')
            except ValueError:
                ds = ds.drop('level')    
    return ds.sel(time=years)


## Data Loading

In [105]:
z_500 = ds.z.sel(level=500).drop('level')
t_850 = ds.t.sel(level=850).drop('level')
t2m = ds.t2m

data = xr.merge([z_500, t_850, t2m])

data_train = data.sel(time=slice('1979', '2016'))
data_test = data.sel(time=slice('2017', '2018'))

z_test = load_test_data('z', ds=ds)
t_test = load_test_data('t', ds=ds)
t2m_test = load_test_data('t', ds=ds)

test_data = xr.merge([z_test, t_test, t2m_test])

_, nlat, nlon = data_train.z.shape; nlat, nlon

Selecting from ds...
Selecting from ds...
Selecting from ds...


(12, 14)

In [88]:
# Compute normalization statistics
# Let's only take a sample of the time to speed this up.
data_mean = data_train.isel(time=slice(0, None, 10000)).mean().load()
data_std = data_train.isel(time=slice(0, None, 10000)).std().load()


# Normalize datasets
data_train = (data_train - data_mean) / data_std
data_test = (data_test - data_mean) / data_std

## Useful functions for Training

In [70]:
def create_training_data(da, lead_time_h, return_valid_time=False):
    """Function to split input and output by lead time."""
    X = da.isel(time=slice(0, -lead_time_h))
    y = da.isel(time=slice(lead_time_h, None))
    valid_time = y.time

    # Must be a "flat" array ==> nlat*nlon
    data = X.values.reshape(-1, nlat*nlon)
    obj = y.values.reshape(-1, nlat*nlon)
    
    if return_valid_time:
        return data, obj, valid_time
    
    return data, obj



def train_regression(model, lead_time_h, input_vars, output_vars, data_subsample=1):
    """Create data, train a linear regression and return the predictions."""
    X_train, y_train, X_test, y_test = [], [], [], []
    for v in input_vars:
        # Create Training Data for a 3 day prediction
        X, y = create_training_data(data_train[v], lead_time_h)
        X_train.append(X)

        if v in output_vars: 
            y_train.append(y)

        # Create Test Data for a 3 day prediction
        X, y, valid_time = create_training_data(data_test[v], lead_time_h, return_valid_time=True)
        X_test.append(X)
        
        if v in output_vars: 
            y_test.append(y)

    X_train, y_train, X_test, y_test = [np.concatenate(d, 1) for d in [X_train, y_train, X_test, y_test]]
    if data_subsample > 1:
        X_train = X_train[::data_subsample]
        y_train = y_train[::data_subsample]
    
    lr = model(n_jobs=16)
    lr.fit(X_train, y_train)
    
    mse_train = mean_squared_error(y_train, lr.predict(X_train))
    mse_test = mean_squared_error(y_test, lr.predict(X_test))

    print(f'Train MSE = {mse_train}')
    print(f'Test MSE = {mse_test}')

    preds = lr.predict(X_test).reshape((-1, len(output_vars), nlat, nlon))
    
    # Save predictions + unnormalize
    preds_ds = []
    for i, v in enumerate(output_vars):
        pred_array = xr.DataArray(
            preds[:, i] * data_std[v].values + data_mean[v].values, 
            dims=['time', 'lat', 'lon'],
            coords={
                'time': valid_time,
                'lat': data_train.lat,
                'lon': data_train.lon
            },
            name=v
        )
        preds_ds.append(pred_array)

    return xr.merge(preds_ds), lr

### Define combinations of training data

In [74]:
# Inputs and Outputs
experiments = [
    [['z'], ['z']],
    [['t'], ['t']],
    [['z', 't'], ['z', 't']],
    [['t2m'], ['t2m']],
    [['z', 't', 't2m'], ['t2m']],
]

## Run Experiments

In [94]:
lead_time = 3*24 # Predict 3 days from current time
preds = []
models = []

for iter, (i, o) in enumerate(experiments):
    print(f'{iter}: Input variables = {i}; output variables = {o}')
    pred, model = train_regression(LinearRegression,
                                   lead_time, 
                                   input_vars=i, 
                                   output_vars=o)
    
    preds.append(pred)
    models.append(model)

    r = compute_weighted_rmse(pred, test_data).compute()
    print('; '.join([f'{v} = {r[v].values}' for v in r]) + '\n')

0: Input variables = ['z']; output variables = ['z']
Train MSE = 784505.6875
Test MSE = 811129.625
z = 164909843.43962523

1: Input variables = ['t']; output variables = ['t']
Train MSE = 11.65665340423584
Test MSE = 12.323047637939453
t = 2903.811498397222

2: Input variables = ['z', 't']; output variables = ['z', 't']
Train MSE = 378123.46875
Test MSE = 400751.5625
z = 164891965.852891; t = 2903.747029536152

3: Input variables = ['t2m']; output variables = ['t2m']
Train MSE = 5.066948890686035
Test MSE = 5.083872318267822


4: Input variables = ['z', 't', 't2m']; output variables = ['t2m']
Train MSE = 4.595192909240723
Test MSE = 4.782581806182861




## Iterative Linear Regression

In [109]:
def create_iterative_lr(state, model, lead_time_h=6, max_lead_time_h=5*24):
    
    max_fc_steps = max_lead_time_h // lead_time_h

    preds_z500, preds_t850 = [], []

    for fc_step in tqdm(range(max_fc_steps)):
        # predict next state and update current with next
        state = model.predict(state)
        
        # Unnormalize
        fc_z500 = state[:, :nlat*nlon].copy() * data_std.z.values + data_mean.z.values
        fc_t850 = state[:, nlat*nlon:].copy() * data_std.t.values + data_mean.t.values
        
        # Reshape
        fc_z500 = fc_z500.reshape((-1, nlat, nlon))
        fc_t850 = fc_t850.reshape((-1, nlat, nlon))
        
        preds_z500.append(fc_z500)
        preds_t850.append(fc_t850)

    return [xr.DataArray(
        np.array(fcs), 
        dims=['lead_time', 'time', 'lat', 'lon'],
        coords={
            'lead_time': np.arange(lead_time_h, max_lead_time_h + lead_time_h, lead_time_h),
            'time': z_test.time,
            'lat': z_test.lat,
            'lon': z_test.lon
        }
    ) for fcs in [preds_z500, preds_t850]]

In [110]:
p, m = train_regression(LinearRegression, 
                        6, 
                        input_vars=['z', 't'], 
                        output_vars=['z', 't'])

Train MSE = 15697.7041015625
Test MSE = 16475.05859375


In [111]:
state = np.concatenate([data_test.z.values.reshape(-1, nlat*nlon), 
                        data_test.t.values.reshape(-1, nlat*nlon)], 1)

In [112]:
fc_z500_6h_iter, fc_t850_6h_iter = create_iterative_lr(state, m)